# 最終課題（メインコンペ） 推論標準コード

## 1.概要
このノートブックは、あなたが学習して Hugging Face にアップした **LoRAアダプタ**を用いて、  
**ベンチマークの推論結果JSONを生成し、コンペの提出用JSONファイルを生成する**ための標準コードです。
- コンペでの提出物は **学習済みLoRAそのものではなく、推論結果のJSONファイル**です。  
- 本ノートは、その提出用JSONを確実に作るための手順を提供します。

- StructEval-Tからサンプリングした150問の回答を生成（推論）します。
- 実行にあたっては、/content/public_150.json（配布資料）が必要です。
- 出力は**inference.json（提出形式）**ですので、これをOmniCampusにアップロードして採点してください。

## 2. 事前準備


- Colab のランタイムを **GPU（T4）** に設定してください。
- Hugging Face にログインします（トークン入力が必要です）。
- 推論に使う LoRA アダプタは、原則として「学習ノートでアップロードしたもの」を使用します。

---



## 3. 実行手順（推奨フロー）



### Step 0: セットアップ（clone / install）
上から順にセルを実行します。

- `StructEval` を clone し、依存関係（vLLM等）を導入します。
- `python3 -m structeval.cli --help` が表示されれば、基本セットアップは成功です。

### Step 1: Hugging Face ログイン
- `login()` を実行し、トークンを入力してください。

### Step 2: LoRA の統合（merge）
- `adapter_id` にある LoRA を読み込み、ベースモデルと統合して `./merged_model` を生成します。
- ここが完了すると、以降の推論は **`./merged_model` をモデルパスとして使用**します。

### Step 3: vLLM 推論の実行と提出用JSONの生成
- `custom_inference.py` が生成され、それを実行します。
- 推論結果は `/content/StructEval/outputs/nonrenderable.json` に保存されます。
- `output` を `generation` に補完し、提出用ファイル `/content/inference.json` を出力します。
- 出力された`/content/inference.json` をダウンロードして、Omnicampusに提出してください。
---



## 4. 出力ファイル（提出物）の扱い



### 4.1 生成される主なファイル
- 統合済みモデル（提出不要）
  - `./merged_model/`

- 推論結果 **提出用ファイル（最重要）**
  - `/content/inference.json`
  - ※このファイルは `generation` フィールドを持つ形式に整形済みです。

### 4.2 提出手順（ダウンロード → Omnicampus にアップロード）
1. Colab 上で、最終成果物 `/content/inference.json` をローカルPCに **ダウンロード**します。
   - Colab 左の「Files（フォルダアイコン）」から `/content/` を開く
   - `inference.json` を右クリック → **Download**

2. Omnicampus の提出画面で、ダウンロードした `inference.json` を **アップロードして提出**します。

提出ファイル名は、`inference.json` としてください。


### **4.3 コンペ参加における注意点**：
- 本コードによる推論には「学習してアップしたLoRA」を使ってください。それ以外のモデルを使った推論結果を提出した方は、失格となります。
- 提出物は「推論結果 JSON」です（LoRA自体の提出ではありません）。
- 提出の際に、HuggingFaceにアップしたアダプタのURLを必ず記載してください。

---

## 5. よくある失敗と対策



- **GPUが有効になっていない**
  - 推論が極端に遅い／vLLMが動かない原因になります。必ず T4 を確認してください。

- **`./merged_model` が存在しない**
  - LoRA統合（merge）が完了していない可能性があります。mergeセルを再実行してください。

- **vLLM 実行時に OOM（Out of Memory）になる**
  - 本標準コードは `gpu_memory_utilization=0.6` で安全寄りですが、環境差で落ちる場合があります。
  - その場合は、まずランタイム再起動（Factory reset）→同じ手順で再実行してください。



---




## 6. 期待する最終状態（チェック）



提出直前に、次を満たしていればOKです。

- `/content/inference.json` が存在する
- そのJSONが list であり、各要素に `generation` フィールドが入っている（空でない）
- Omnicampus に `inference.json` をアップロードして提出
---


# 実行コード


### Step 0: セットアップ（clone / install）

In [ ]:
# 0) Setup (バージョン固定)

!git clone -b fix-module-not-found-issue-2 https://github.com/Osakana7777777/StructEval.git

!uv pip install \
  "vllm==0.13.0" \
  "torch==2.9.0" \
  "torchaudio==2.9.0" \
  "torchvision==0.24.0" \
  "triton==3.5.0" \
  "compressed-tensors==0.12.2" \
  "openai==2.15.0" \
  "xgrammar==0.1.27" \
  "bitsandbytes==0.46.1" \
  fire

# flash-attn だけは環境によって挙動が変わるためバージョン固定しない
!uv pip install flash-attn --no-build-isolation

%cd StructEval
!uv pip install -e .

!python3 -m structeval.cli --help
!mkdir -p outputs


Cloning into 'StructEval'...
remote: Enumerating objects: 17398, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 17398 (delta 91), reused 45 (delta 26), pack-reused 17249 (from 3)
Receiving objects: 100% (17398/17398), 529.90 MiB | 16.40 MiB/s, done.
Resolving deltas: 100% (5424/5424), done.
Using Python 3.12.12 environment at: /usr
Resolved 165 packages in 1.92s
Prepared 47 packages in 20.14s
Uninstalled 4 packages in 118ms
Installed 47 packages in 172ms
 + anthropic==0.71.0
 + apache-tvm-ffi==0.1.8.post2
 + astor==0.8.1
 + bitsandbytes==0.46.1
 + blake3==1.0.8
 + cbor2==5.8.0
 + compressed-tensors==0.12.2
 + depyf==0.20.0
 + diskcache==5.6.3
 + dnspython==2.8.0
 + email-validator==2.3.0
 + fastapi-cli==0.0.20
 + fastapi-cloud-cli==0.11.0
 + fastar==0.8.0
 + fire==0.7.1
 + flashinfer-python==0.5.3
 + gguf==0.17.1
 + ijson==3.4.0.post0
 + interegular==0.3.3
 + jmespath==1.1.0
 - lark==1.3.1
 + lark==1.2.2
 + llguida


### Step 1: Hugging Face ログイン
- `login()` を実行し、トークンを入力してください。


In [ ]:

# -----------------------------
# 1) HF login (once)
# -----------------------------
# HF Hub上のデータセットを読むため、HuggingFaceにログインします。
#
from huggingface_hub import login
login()  # Colab will prompt


### Step 2: LoRA の統合（merge）
- `adapter_id` にある LoRA を読み込み、ベースモデルと統合して `./merged_model` を生成します。
- ここが完了すると、以降の推論は **`./merged_model` をモデルパスとして使用**します。



- ここで、contentフォルダに"public_150.json"をアップロードしてください。
- Colabのファイル領域(/content)に、評価用の public_150.json を置く必要があります。

In [ ]:
# ------------------------------------------------------------
# 1) Config
# ------------------------------------------------------------

MODEL_SOURCE = "adapter_merge"   # "merged" | "base" | "adapter_merge"
# どのモデルを使うかを選びます。今回は、基本的に"adapter_merge"を選んでください。

#   - "base"        : ベースモデル（学習していない素のモデル）
#   - "merged"      : すでにLoRAをマージ済みのモデル（完成品として配布されている想定）
#   - "adapter_merge": ベースモデル + LoRAアダプタをその場で読み込み、ローカルでマージしてから使う

# base model (HF repo id or local path)
# 学習時に使用したベースモデルを入れてください。
BASE_MODEL_ID_OR_PATH   = "Qwen/Qwen3-4B-Instruct-2507"

# merged model (HF repo id or local path)
# アダプタではなくマージモデルをアップロードした場合は、ここにIDをいれてください。
# "merged"を選択した場合に記入
MERGED_MODEL_ID_OR_PATH = "your_id/your-merged-repo"

# adapter merge
# あなたがHuggingFaceにアップロードしたアダプタのIDを入れてください。
# "adapter_merge"を選択した場合に記入
ADAPTER_ID       = "your_id/test-lora-repo"

# merge済モデルの一時保存
MERGED_LOCAL_DIR = "./merged_model"

# 入力（150問）と出力（提出用）ファイルパスの指定
INPUT_PATH  = "/content/public_150.json"
OUTPUT_PATH = "/content/inference.json"


TEMPERATURE = 0.0
#   0.0 は最も決定的（同じ入力なら同じ出力になりやすい）で、評価用途では一般に安定します。


### Step 3: vLLM 推論の実行と提出用JSONの生成
- `custom_inference.py` が生成され、それを実行します。
- 推論結果は `/content/StructEval/outputs/nonrenderable.json` に保存されます。
- `output` を `generation` に補完し、提出用ファイル `/content/inference.json` を出力します。
- 出力された`/content/inference.json` をダウンロードして、Omnicampusに提出してください。
---

In [ ]:

# ------------------------------------------------------------
# 2) Stable vLLM env (IMPORTANT: must be set BEFORE importing vllm)
# ------------------------------------------------------------

import os
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
# vLLM内部でワーカープロセスを作る方式を "spawn" に固定します。
# Colabなど一部環境では "fork" より安定しやすいことがあります。

os.environ["VLLM_LOGGING_LEVEL"] = "INFO"
# vLLMのログレベル（INFO）を設定します。デバッグ時に有用です。

# ------------------------------------------------------------
# 3) Resolve model_path
# ------------------------------------------------------------
# 選んだMODEL_SOURCEに応じて、最終的にvLLMに渡す「モデルの場所(model_path)」を決めます。

def resolve_model_path():
    # どのモデルを使うかに応じて、vLLMへ渡すパス/IDを返す関数

    if MODEL_SOURCE == "base":
        return BASE_MODEL_ID_OR_PATH

    if MODEL_SOURCE == "merged":
        return MERGED_MODEL_ID_OR_PATH

    if MODEL_SOURCE == "adapter_merge":
        # NOTE: torch/CUDA（GPU）を触るため、vLLMを起動する前に済ませます。
        import os, gc
        import torch
        from transformers import AutoModelForCausalLM, AutoTokenizer
        from peft import PeftModel
        print("[INFO] Merging adapter into base model...")
        base_model = AutoModelForCausalLM.from_pretrained(
            BASE_MODEL_ID_OR_PATH,
            dtype=torch.float16,
            device_map="auto",
            trust_remote_code=True,
        )
        # ベースモデルに対応するトークナイザを読み込み（マージ後も同じものを使うのが通常）
        tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID_OR_PATH, trust_remote_code=True)

        # base_model に LoRAアダプタ(ADAPTER_ID) をマージ
        # merge後はLoRA層を外せるので（unload）、推論時の扱いが単純になります。
        model_to_merge = PeftModel.from_pretrained(base_model, ADAPTER_ID)
        merged_model = model_to_merge.merge_and_unload()

        os.makedirs(MERGED_LOCAL_DIR, exist_ok=True)
        merged_model.save_pretrained(MERGED_LOCAL_DIR)
        tokenizer.save_pretrained(MERGED_LOCAL_DIR)

        del base_model, model_to_merge, merged_model
        gc.collect()
        torch.cuda.empty_cache()
        print("[INFO] Merged model saved:", MERGED_LOCAL_DIR)
        return MERGED_LOCAL_DIR

    raise ValueError("MODEL_SOURCE must be 'merged'|'base'|'adapter_merge'")

# 最終的に使うモデルのパス/IDを確定
model_path = resolve_model_path()
print("[INFO] Using model:", model_path)

# ------------------------------------------------------------
# 4) Load public_150 and build prompts (no torch usage here)
# ------------------------------------------------------------
# 入力ファイルを読み込み、各問題の「プロンプト（モデルに渡す文字列）」を作ります。

import json
from pathlib import Path
from transformers import AutoTokenizer

pub = json.loads(Path(INPUT_PATH).read_text(encoding="utf-8"))

assert isinstance(pub, list), "public_150.json must be a list"
assert len(pub) == 150, f"public_150 must have 150 items, got {len(pub)}"
assert len({x["task_id"] for x in pub}) == 150, "public_150 has duplicate task_id"

# Safety: ensure output_type exists (office enriched file)

missing_ot = [x.get("task_id") for x in pub if not (x.get("output_type") or "").strip()]

if missing_ot:
    raise RuntimeError(f"FATAL: public_150 missing output_type (not enriched). Examples: {missing_ot[:5]}")

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# task_ids: 出力に使う task_id の並びを保存
# prompts:   vLLMに渡すプロンプト文字列を保存
task_ids, prompts = [], []

for item in pub:
    task_ids.append(item["task_id"])
    query = item.get("query", "")
    messages = [{"role": "user", "content": query}]
    prompts.append(tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True))
    # ↑ apply_chat_template で「モデルが期待する会話形式の文字列」に整形
    #   tokenize=False : まだトークン化せず、文字列として返す
    #   add_generation_prompt=True : 「ここからアシスタントが答える」境界を追加
    #   これにより、モデルが回答を続けて生成しやすい形になります。

# ------------------------------------------------------------
# 5) Presets + fallback plan
# ------------------------------------------------------------
# vLLM起動時に「文脈長(max_model_len)」や「出力上限(max_tokens)」を大きくしすぎると、
# GPUメモリ不足(OOM)で落ちやすいです。
# そこで、成功しやすい設定をいくつか用意し、失敗したら段階的に軽くして再試行します。
# merged（既に焼き込み済み）と adapter_merge（その場でマージ）では、
# 実メモリ使用量が変わることがあるため、最初に試す設定（gpu_memなど）を変えています。
# 事前に「試行候補リスト」を作り、上から順に試します。

def build_try_configs():

    # Primary presets

    if MODEL_SOURCE == "merged":
        base = [
            {"max_model_len": 4096, "max_tokens": 4096, "gpu_mem": 0.85},
            {"max_model_len": 4096, "max_tokens": 4096, "gpu_mem": 0.80},
        ]
        # ↑ 4096トークンまでの文脈/出力を許しつつ、GPU使用率を0.85→0.80で試す

    elif MODEL_SOURCE == "adapter_merge":
        base = [
            {"max_model_len": 4096, "max_tokens": 4096, "gpu_mem": 0.60},
            {"max_model_len": 4096, "max_tokens": 4096, "gpu_mem": 0.65},
        ]
        # ↑ adapter_merge はメモリが厳しくなりがちなので、gpu_memを低めから試します。

    else:  # base
        base = [
            {"max_model_len": 4096, "max_tokens": 4096, "gpu_mem": 0.80},
            {"max_model_len": 4096, "max_tokens": 4096, "gpu_mem": 0.70},
        ]
        # ↑ baseモデルは比較的軽い想定で、0.80→0.70を試します。

    # Fallback ladder (reduce context / output)
    # 失敗したときの「段階的に軽くする設定」。
    # max_model_len と max_tokens を下げると、必要メモリが減り成功しやすくなります。
    ladder = [
        {"max_model_len": 3072, "max_tokens": 3072},
        {"max_model_len": 2048, "max_tokens": 2048},
        {"max_model_len": 1536, "max_tokens": 1536},
    ]

    # Expand base configs with ladder and a couple gpu_mem tweaks
    # ↑ base設定に対し、ladder段階を「合成」して試行パターンを増やします。
    #   また、gpu_memも少し増やす版を試します（失敗理由が「確保不足」系のときに効く場合がある）。
    out = []
    for cfg in base:
        out.append(cfg)

        for step in ladder:
            out.append({**cfg, **step})

        # try a slightly higher gpu_mem if still failing (some failures are "not enough alloc")
        out.append({**cfg, "gpu_mem": min(0.90, cfg["gpu_mem"] + 0.05)})

    # Deduplicate while preserving order
    # ↑ 似た設定が重複し得るので、順序を保ったまま重複削除します。
    seen = set()
    uniq = []
    for c in out:
        key = (c["max_model_len"], c["max_tokens"], round(c["gpu_mem"], 2))

        if key in seen:
            continue

        seen.add(key)
        uniq.append(c)

    return uniq


TRY_CONFIGS = build_try_configs()
# ↑ 実際に試す設定リストを作成します。

print("[INFO] Try configs (in order):")

for i, c in enumerate(TRY_CONFIGS[:8], 1):
    print(f"  {i:02d}. max_model_len={c['max_model_len']} max_tokens={c['max_tokens']} gpu_mem={c['gpu_mem']}")

if len(TRY_CONFIGS) > 8:
    print(f"  ... total {len(TRY_CONFIGS)} configs")

# ------------------------------------------------------------
# 6) vLLM run with retry
# ------------------------------------------------------------
# ↑ ここからが推論本体です。

from vllm import LLM, SamplingParams
def run_with_config(cfg):

    sampling = SamplingParams(
        temperature=TEMPERATURE,
        max_tokens=cfg["max_tokens"],
    )

    llm = LLM(
        model=model_path,
        max_model_len=cfg["max_model_len"],
        gpu_memory_utilization=cfg["gpu_mem"],
        enforce_eager=True,
        tensor_parallel_size=1,
         disable_log_stats=True,
    )

    outs = llm.generate(prompts, sampling)

    submission = []
    # ↑ 提出形式 [{"task_id": ..., "generation": ...}, ...] を作ります。

    for tid, out in zip(task_ids, outs):
        gen = out.outputs[0].text if out.outputs else ""
        submission.append({"task_id": tid, "generation": gen})
    return submission
    # ↑ 150問ぶんの提出配列を返します。

last_err = None
submission = None
# ↑ 成功した場合に提出データ（150件）を入れる変数。成功まではNone。

for idx, cfg in enumerate(TRY_CONFIGS, 1):
    print(f"[INFO] Attempt {idx}/{len(TRY_CONFIGS)}: max_model_len={cfg['max_model_len']} max_tokens={cfg['max_tokens']} gpu_mem={cfg['gpu_mem']}")
    try:
        submission = run_with_config(cfg)
        print("[INFO] ✅ Generation succeeded with this config.")
        # ↑ 成功ログ
        break
    except RuntimeError as e:
        last_err = e
        msg = str(e)
        print("[WARN] Failed:", msg[:200].replace("\n", " "))

# try next config
if submission is None:
    raise RuntimeError(f"All configs failed. Last error: {last_err}")


# Final guards
# ↑ 最後に「提出物としての整合性チェック」をします。

if len(submission) != 150:
    # ↑ 150件生成できているかチェック
    raise RuntimeError(f"Submission count mismatch: {len(submission)}")

if len({x['task_id'] for x in submission}) != 150:
    # ↑ task_id の重複がないかチェック
    raise RuntimeError("Duplicate task_id in submission")

Path(OUTPUT_PATH).write_text(json.dumps(submission, ensure_ascii=False, indent=2), encoding="utf-8")
# ↑ submission（Pythonオブジェクト）をJSON文字列にしてファイルへ保存します。

print("[OK] wrote:", OUTPUT_PATH, "items=150")
